<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

**Описание проекта**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель проекта**

Требуется обучить модель классифицировать комментарии на позитивные и негативные.

Построить модель со значением метрики качества *F1* не меньше 0.75. 

**План проекта**

1. Загрузить и подготовьть данные.
2. Обучить разные модели. 
3. Сделайть выводы.

Для выполнения проекта применять *BERT* необязательно, но можно попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('omw-1.4')
import numpy as np
import pandas as pd
import re 

#from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from nltk.stem import WordNetLemmatizer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from tqdm import tqdm


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Adelina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Adelina\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Adelina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Adelina\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Загрузим данные и изучим данные.

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=0)


In [3]:
data.head(15)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Лемматизируем содержание столбца 'text' и очистим текст. 

In [5]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text) 
    lemm_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])  
    return lemm_text

def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text) 
    text = text.split()
    return " ".join(text).lower() 

In [6]:
tqdm.pandas()
data['text'] = data['text'].progress_apply(clear_text)
data

100%|███████████████████████████████████████████████████████████████████████| 159292/159292 [00:03<00:00, 41019.87it/s]


,text,toxic
0,explanation why the edits made under my userna...,0
1,d aww he matches this background colour i m se...,0
2,hey man i m really not trying to edit war it s...,0
3,more i can t make any real suggestions on impr...,0
4,you sir are my hero any chance you remember wh...,0
...,...,...
159446,and for the second time of asking when your vi...,0
159447,you should be ashamed of yourself that is a ho...,0
159448,spitzer umm theres no actual article for prost...,0
159449,and it looks like it was actually you who put ...,0


In [8]:
data['text'] = data['text'].progress_apply(lemmatize)
data

100%|████████████████████████████████████████████████████████████████████████| 159292/159292 [3:25:40<00:00, 12.91it/s]


,text,toxic
0,explanation why the edits make under my userna...,0
1,d aww he match this background colour i m seem...,0
2,hey man i m really not try to edit war it s ju...,0
3,more i can t make any real suggestion on impro...,0
4,you sir be my hero any chance you remember wha...,0
...,...,...
159446,and for the second time of ask when your view ...,0
159447,you should be ashamed of yourself that be a ho...,0
159448,spitzer umm there no actual article for prosti...,0
159449,and it look like it be actually you who put on...,0


Сохраним лемматизированные данные.

In [20]:
#data.to_csv('lemm_test')

Удалим пропуски.

In [9]:
data = data.dropna()

Разобьем данные на обучающую и тестовую выборку.

In [10]:
train, test = train_test_split(data,  test_size=0.3, random_state = 12345, shuffle=True, stratify=data['toxic'])

Разделим данные на целевой признак и остальные.

In [11]:
features_train = train.drop(['toxic'],axis=1)
target_train = train['toxic']
features_test = test.drop(['toxic'],axis=1)
target_test = test['toxic']

Проверим соотношение классов.

In [12]:
train.loc[data['toxic'] == 0].shape[0]/train.loc[data['toxic'] == 1].shape[0]

8.841482789055604

У признака "toxic" 0 почти в 9 раза больше, чем 1.

**Вывод:**

1. Были изучены данные.
2. Лемматизировано содержание столбца 'text' и очищено текст. 
3. Выявлен дисбаланс классов. У признака "toxic" значение 0 встречалось почти в 9 раза больше, чем 1.

## Обучение

Построим модель с использованием алгоритма дерева решений.

In [16]:
features_train = features_train['text'].values
features_test = features_test['text'].values

#Установка стоп слов
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adelina\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [17]:
pipeline = Pipeline([
    ('vectorazer', TfidfVectorizer(stop_words=stopwords)),

    ('model', DecisionTreeClassifier(random_state=12345,class_weight="balanced"))])

params = {'model__max_depth':np.arange(1,11)}

search_DecisionTreeClassifier = GridSearchCV(pipeline, cv=5, n_jobs=-1, param_grid=params ,scoring='f1')
search_DecisionTreeClassifier.fit(features_train,target_train)
print(search_DecisionTreeClassifier.best_estimator_)
print(search_DecisionTreeClassifier.best_score_)

Pipeline(steps=[('vectorazer',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 DecisionTreeClassifier(class_weight='balanced', max_depth=10,
                                        random_state=12345))])
0.5780868711517251


Построим модель с использованием алгоритма случайного леса.

In [21]:
pipeline = Pipeline([
    ('vectorazer', TfidfVectorizer(stop_words=stopwords)),
    ('model', RandomForestClassifier(random_state=12345,class_weight="balanced"))])

params = {'model__max_depth':np.arange(1,7),
         'model__n_estimators':np.arange(20, 40, 10)}

search_RandomForestClassifier = GridSearchCV(pipeline, scoring='f1', param_grid = params, cv = 3)
search_RandomForestClassifier.fit(features_train,target_train)
print(search_RandomForestClassifier.best_estimator_)
print(search_RandomForestClassifier.best_score_)

Pipeline(steps=[('vectorazer',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 RandomForestClassifier(class_weight='balanced', max_depth=6,
                                        n_estimators=30, random_state=12345))])
0.3066626696941494


Построим модель с использованием линейной регрессии.

In [22]:
pipeline = Pipeline([
    ('vectorazer', TfidfVectorizer(stop_words=stopwords)),
    ('model', LogisticRegression(random_state=12345,class_weight="balanced"))])

params = {'model__C':np.arange(1,20)}

search_LogisticRegression = GridSearchCV(pipeline, scoring='f1', param_grid = params, cv = 3)
search_LogisticRegression.fit(features_train,target_train)
print(search_LogisticRegression.best_estimator_)
print(search_LogisticRegression.best_score_)

C:\Users\Adelina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Adelina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

Pipeline(steps=[('vectorazer',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 LogisticRegression(C=13, class_weight='balanced',
                                    random_state=12345))])
0.7569001852502146


C:\Users\Adelina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
pipeline = Pipeline([
    ('vectorazer', TfidfVectorizer(stop_words=stopwords)),
    ('model', LGBMClassifier(random_state=12345,class_weight="balanced"))])

params = {'model__max_depth':np.arange(1,7),
         'model__n_estimators':np.arange(20, 40, 10)}

search_LGBMClassifier = GridSearchCV(pipeline, scoring='f1', param_grid = params, cv = 3)
search_LGBMClassifier.fit(features_train,target_train)
print(search_LGBMClassifier.best_estimator_)
print(search_LGBMClassifier.best_score_)

Pipeline(steps=[('vectorazer',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('model',
                 LGBMClassifier(class_weight='balanced', max_depth=6,
                                n_estimators=30, random_state=12345))])
0.6696892387657316


Наилучший результат показала модель LogisticRegression(C=13, class_weight='balanced',
                                    random_state=12345).

**Вывод:**

Были проанализированы модели с использованием алгоритма дерева решений, с использованием алгоритма случайного леса и линейной регрессии в связке GridSearchCV + pipeline. Наилучший результат f1 показала модель LogisticRegression(C=13, class_weight='balanced', random_state=12345).

## Тестирование

Протестируем модель на тестовых данных.

In [24]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
features_train = count_tf_idf.fit_transform(features_train) 
features_test = count_tf_idf.transform(features_test) 

In [26]:
model = LogisticRegression(C=13, class_weight='balanced',
                                    random_state=12345)
model.fit(features_train,target_train)
prediction = model.predict(features_test)
f1_score(target_test,prediction)

C:\Users\Adelina\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7594936708860758

**Вывод:**
Модель LogisticRegression(C=13, class_weight='balanced', random_state=12345 показала на тестовых данных результат f1 = 0.76.

## Выводы

1. Подготовка

    1.1. Были изучены данные.
    
    1.2. Лемматизировано содержание столбца 'text' и очищено текст. 
    
    1.3. Выявлен дисбаланс классов. У признака "toxic" значение 0 встречалось почти в 9 раза больше, чем 1.
    
    
2. Были проанализированы модели с использованием алгоритма дерева решений, с использованием алгоритма случайного леса и линейной регрессии в связке GridSearchCV + pipeline. Наилучший результат f1 показала модель LogisticRegression(C=13, class_weight='balanced', random_state=12345).


3. Модель LogisticRegression(C=13, class_weight='balanced', random_state=12345 показала на тестовых данных результат f1 = 0.76.